# Challenge MIASHS 2025 – Identification Automatique des Collemboles

Ce projet s'inscrit dans le cadre du Challenge MIASHS 2025 et a pour objectif de développer un modèle de deep learning capable d'identifier automatiquement les espèces de collemboles (petits arthropodes du sol), à partir d’images annotées par des experts.

## Données

Le dossier `data/` contient :
- Un ensemble d’images de collemboles, extraites de divers protocoles de collecte (ex : `TIDM_URBA`, `DIJON2021`, etc.).
- Un fichier d’annotations, dans lequel chaque ligne contient :
  - Les votes de 4 experts sur l’espèce, sous la forme `3_3_3_3` (chaque chiffre représentant l’espèce choisie par un expert).

  - Les coordonnées normalisées de la bounding box : `x_center y_center width height`.

Exemple d’annotation :
3_3_3_3 TIDM_URBA/DIJON2021_2/3 0.3294 0.7812 0.3158 0.4365



## Modèle 1: EfficientNet-B0 fine tuné

In [52]:
# Cellule 1: Importation des bibliothèques et définition des paramètres
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




In [53]:
# Chemins et paramètres
data_dir = "/kaggle/input/data-train-unanimes-finales"  # Dossier contenant les images
csv_path = "/kaggle/input/annotations-unanimes-finales/annotations_unanimes.csv"  # CSV avec colonnes: id, espece
target_size = (224,224)
batch_size = 4
num_epochs = 50

In [54]:
# Cellule 2: Chargement du CSV et préparation de la DataFrame
df = pd.read_csv(csv_path)
# Créer une colonne "filename" en ajoutant ".jpg" au champ "id"
df["filename"] = df["id"] + ".jpg"
# Convertir la colonne "espece" en chaîne de caractères (pour le générateur)
df["espece"] = df["espece"].astype(str)



In [55]:
df.head()

,id,espece,filename
0,0.9096178742684140.31465763459794080.936818297...,0,0.9096178742684140.31465763459794080.936818297...
1,0.9096178742684140.31465763459794080.936818297...,8,0.9096178742684140.31465763459794080.936818297...
2,0.73056986349619260.72325076210210060.07719147...,0,0.73056986349619260.72325076210210060.07719147...
3,0.73056986349619260.72325076210210060.07719147...,8,0.73056986349619260.72325076210210060.07719147...
4,0.9187565506779370.52926796120235030.838969372...,0,0.9187565506779370.52926796120235030.838969372...


In [56]:
# Cellule 3: Split train/validation et création des générateurs d'images

from sklearn.model_selection import train_test_split

# Séparation en 80% train / 20% validation, en stratifiant sur "espece"
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["espece"])

# Data augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    brightness_range=(0.8, 1.2),
    zoom_range=(1.0, 1.2)
)
# Pour la validation, on se contente du rescale
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=data_dir,
    x_col="filename",
    y_col="espece",
    target_size=target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory=data_dir,
    x_col="filename",
    y_col="espece",
    target_size=target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_generator.class_indices)
print("Nombre de classes :", num_classes)


Found 1037 validated image filenames belonging to 9 classes.
Found 260 validated image filenames belonging to 9 classes.
Nombre de classes : 9


In [57]:
# Cellule 4: Construction du modèle
# Charger EfficientNetB0 avec les poids ImageNet, sans la tête de classification
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(target_size[0], target_size[1], 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Pour réentraîner toutes les couches, on s'assure que toutes les couches sont entraînables
for layer in base_model.layers:
    layer.trainable = True

model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_12 (Rescaling)  │ (None, 224, 224, 3)    │              0 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_6           │ (None, 224, 224, 3)    │              7 │ rescaling_12[0][0]     │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_13 (Rescaling)  │ (None, 224, 224, 3)    │              0 │ normalization_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_13[0][0]     │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 4,061,100 (15.49 MB)

 Trainable params: 4,019,077 (15.33 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [58]:
# Cellule 5: Définition du callback F1Macro et entraînement initial

from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import numpy as np

# Callback pour calculer le F1 score macro à la fin de chaque époque
class F1MacroCallback(Callback):
    def __init__(self, val_generator):
        super().__init__()
        self.val_generator = val_generator

    def on_epoch_end(self, epoch, logs=None):
        # Nombre de batches dans le générateur de validation
        steps = len(self.val_generator)
        # Prédire sur l'ensemble du jeu de validation
        preds = self.model.predict(self.val_generator, steps=steps)
        # Convertir les prédictions en indices de classes
        val_predict = np.argmax(preds, axis=1)
        # Récupérer les véritables labels dans le même ordre
        val_true = self.val_generator.classes  
        # Calculer le F1 macro
        f1 = f1_score(val_true, val_predict, average='macro')
        print(f"\nF1 Score Macro (val) = {f1:.4f}")
        logs["val_f1_macro"] = f1

# Instancier le callback F1
f1_callback = F1MacroCallback(val_generator)

# Définir les autres callbacks (EarlyStopping et ModelCheckpoint)
callbacks = [
    EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True, verbose=1),
    f1_callback
]

# Compilation du modèle pour l'entraînement initial
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)



In [ ]:
# Entraînement du modèle (Phase 1)
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/steptep - accuracy

F1 Score Macro (val) = 0.0159
260/260 ━━━━━━━━━━━━━━━━━━━━ 129s 246ms/step - accuracy: 0.3859 - loss: 1.7463 - val_accuracy: 0.0769 - val_loss: 8.0685 - val_f1_macro: 0.0159
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/stepep - accurac

F1 Score Macro (val) = 0.1640
260/260 ━━━━━━━━━━━━━━━━━━━━ 26s 97ms/step - accuracy: 0.4949 - loss: 1.3929 - val_accuracy: 0.2577 - val_loss: 3.4510 - val_f1_macro: 0.1640
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/stepep - accuracy

F1 Score Macro (val) = 0.5910
260/260 ━━━━━━━━━━━━━━━━━━━━ 25s 94ms/step - accuracy: 0.5420 - loss: 1.2492 - val_accuracy: 0.5962 - val_loss: 1.1191 - val_f1_macro: 0.5910
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/stepep - accuracy:

F1 Score Macro (val) = 0.6051
260/260 ━━━━━━━━━━━━━━━━━━━━ 25s 92ms/step - accuracy: 0.6082 - loss: 1.0116 - val_accuracy: 0.6731 - val_loss: 1.0396 - val_f1_macro: 0.6051
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/stepep - accur